In [66]:
import pandas as pd
data_path = "swiggy.csv"
df = pd.read_csv(data_path)

df['review_date'] = pd.to_datetime(df['review_date'])  # ensure it's datetime
df = df.sort_values(by='review_date', ascending=False)
# Step 1: Convert review_date to datetime
df['review_date'] = pd.to_datetime(df['review_date'])

# Step 2: Extract only the date part
df['date'] = df['review_date'].dt.date


In [67]:
df = df[['date','review_description','rating']]

In [68]:
df.head()

,date,review_description,rating
59992,2023-07-24,Very fantastic aap,5
3313,2023-07-24,Wonderful discounts & cheap and best and user ...,5
3703,2023-07-24,This delivery app makes easy for us to order i...,5
5508,2023-07-24,Only half order sent never order from this Dom...,1
62135,2023-07-24,"Fedup of message: ""Your order is picked up wit...",1


In [69]:
# end_date = pd.to_datetime('2022-07-31').date()
# start_date = end_date - pd.Timedelta(days=30)

# df_filtered = df[(df['date'] >= start_date) & (df['date'] <= end_date)]


In [70]:
# df_filtered.head()

In [71]:
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module='tqdm')

for i in tqdm(range(100)):
    pass



100%|██████████| 100/100 [00:00<00:00, 340723.31it/s]


In [92]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from collections import defaultdict, deque
import numpy as np

model = SentenceTransformer('all-MiniLM-L6-v2')
# model = SentenceTransformer("all-mpnet-base-v2")

class TopicLimiterReviewTrendAgent:
    def __init__(self, seed_topics, similarity_threshold=0.75, window_days=31, max_topics=20):
        self.similarity_threshold = similarity_threshold
        self.window_days = window_days
        self.max_topics = max_topics
        self.topic_phrases = {}
        self.topic_centroids = {}
        self.topic_counter = 0
        for topic, phrases in seed_topics.items():
            self.topic_phrases[topic] = phrases.copy()
            embeddings = model.encode(phrases)
            self.topic_centroids[topic] = np.mean(embeddings, axis=0)
        self.topics = list(seed_topics.keys())
        self.topic_history = defaultdict(lambda: defaultdict(int))
        self.dates_queue = deque(maxlen=self.window_days)
    
    def assign_topic(self, review_embedding):
        centroids = np.vstack([self.topic_centroids[t] for t in self.topics])
        sims = cosine_similarity([review_embedding], centroids)[0]
        max_idx, max_sim = np.argmax(sims), np.max(sims)
        if max_sim >= self.similarity_threshold:
            return self.topics[max_idx], max_sim
        else:
            return None, max_sim

    def merge_closest_topics(self):
        # Find pair with max centroid similarity among discovered topics (excluding seed topics if desired)
        topics_to_consider = [t for t in self.topics if t not in self.topic_phrases or t.startswith("new_topic")]
        if len(topics_to_consider) < 2:
            return  # Nothing to merge
        centroids = [self.topic_centroids[t] for t in topics_to_consider]
        sims = cosine_similarity(centroids)
        np.fill_diagonal(sims, -1)  # Don't compare topic to itself
        idx1, idx2 = np.unravel_index(np.argmax(sims), sims.shape)
        topic1, topic2 = topics_to_consider[idx1], topics_to_consider[idx2]
        # Merge topic2 into topic1
        self.topic_phrases[topic1].extend(self.topic_phrases[topic2])
        self.update_centroid(topic1)
        for date, count in self.topic_history[topic2].items():
            self.topic_history[topic1][date] += count
        del self.topic_phrases[topic2]
        del self.topic_centroids[topic2]
        del self.topic_history[topic2]
        self.topics.remove(topic2)
    
    def process_daily_batch(self, date, reviews):
        self.dates_queue.append(date)
        for review in reviews:
            emb = model.encode(review)
            assigned_topic, sim = self.assign_topic(emb)
            if assigned_topic is not None:
                self.topic_phrases[assigned_topic].append(review)
                self.update_centroid(assigned_topic)
                self.topic_history[assigned_topic][date] += 1
            else:
                # Create new topic only if under limit, otherwise merge first
                if len(self.topics) >= self.max_topics:
                    self.merge_closest_topics()
                self.topic_counter += 1
                snippet = " ".join(review.split()[:6])
                new_topic = f"new_topic {self.topic_counter}: {snippet}"
                self.topics.append(new_topic)
                self.topic_phrases[new_topic] = [review]
                self.topic_centroids[new_topic] = emb
                self.topic_history[new_topic][date] = 1
        # Purge rolling window
        cutoff_dates = set(self.dates_queue)
        for topic in self.topic_history:
            for d in list(self.topic_history[topic].keys()):
                if d not in cutoff_dates:
                    del self.topic_history[topic][d]
    
    def update_centroid(self, topic):
        embeddings = model.encode(self.topic_phrases[topic])
        self.topic_centroids[topic] = np.mean(embeddings, axis=0)
    
    def get_trend_report(self, target_date):
        target_date = pd.to_datetime(target_date).date()
        dates_window = [target_date - pd.Timedelta(days=i) for i in reversed(range(self.window_days))]
        data = []
        for topic in self.topics:
            row = [self.topic_history[topic].get(d, 0) for d in dates_window]
            data.append([topic] + row)
        columns = ['Topic'] + [str(d) for d in dates_window]
        df_trend = pd.DataFrame(data, columns=columns).set_index('Topic')
        return df_trend


In [111]:
seed_topics = {
    "Delivery issue": ["Delivery delayed", "Order late", "Not delivered", "Order cancelled","late delivery"],
    "Food stale": ["Stale food", "Food not fresh", "Spoiled food","missing food"],
    "Delivery partner rude": ["Delivery guy was rude", "Delivery partner misbehaved", "Delivery person was impolite","Delivery boy was very bad"],
    "Maps not working properly": ["Maps issue", "Map not loading", "GPS problem"],
    "Instamart should be open all night": ["Instamart should be open all night", "Keep Instamart open always"],
    "Bring back 10 minute bolt delivery": ["Bring back 10 minute bolt delivery", "Miss 10 min orders"],
    "Payment issue": ["Payment failed", "Couldn't pay", "Payment not accepted"],
    "Customer support unresponsive": ["Customer support not responding", "No help from support"],
}

agent = TopicLimiterReviewTrendAgent(
    seed_topics=seed_topics,
    similarity_threshold=0.7)

In [74]:
import datetime

In [75]:
start_date = df['date'].min()
end_date = df['date'].max()

print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2018-09-12
End Date: 2023-07-24


In [76]:
end_date = pd.to_datetime('2022-08-31').date()
start_date = end_date - pd.Timedelta(days=30)

df_filtered = df[(df['date'] >= start_date) & (df['date'] <= end_date)]

In [96]:
df_sampled = (
    df_filtered
    .groupby('date', group_keys=False)
    .apply(lambda x: x.sample(n=10, random_state=42) if len(x) >= 10 else x)
    .reset_index(drop=True)
)

In [97]:
start_date = df_sampled['date'].min()
end_date = df_sampled['date'].max()

print("Start Date:", start_date)
print("End Date:", end_date)

Start Date: 2022-08-01
End Date: 2022-08-31


In [98]:
df_2 = df_sampled.copy()

In [99]:
df_2.shape

(310, 3)

In [100]:
df3 = df_2

In [101]:
daily_data = df3.groupby('date')['review_description'].apply(list).to_dict()

In [102]:
pd.set_option("display.max_columns",None)

In [103]:
daily_data.keys()

dict_keys([datetime.date(2022, 8, 1), datetime.date(2022, 8, 2), datetime.date(2022, 8, 3), datetime.date(2022, 8, 4), datetime.date(2022, 8, 5), datetime.date(2022, 8, 6), datetime.date(2022, 8, 7), datetime.date(2022, 8, 8), datetime.date(2022, 8, 9), datetime.date(2022, 8, 10), datetime.date(2022, 8, 11), datetime.date(2022, 8, 12), datetime.date(2022, 8, 13), datetime.date(2022, 8, 14), datetime.date(2022, 8, 15), datetime.date(2022, 8, 16), datetime.date(2022, 8, 17), datetime.date(2022, 8, 18), datetime.date(2022, 8, 19), datetime.date(2022, 8, 20), datetime.date(2022, 8, 21), datetime.date(2022, 8, 22), datetime.date(2022, 8, 23), datetime.date(2022, 8, 24), datetime.date(2022, 8, 25), datetime.date(2022, 8, 26), datetime.date(2022, 8, 27), datetime.date(2022, 8, 28), datetime.date(2022, 8, 29), datetime.date(2022, 8, 30), datetime.date(2022, 8, 31)])

In [105]:
print(daily_data[datetime.date(2022, 8, 3)])

["I am a swiggy one member and Yesterday I ordered food. The packaging was open and food items were missing i reported this issue. However, my tickets were force closed without any resolution. No one contacted me back. It's a hard earned money I never had this much issues with Zomato. Very much disappointed. Still I am not able to get any option to chal or talk to any representative. Pathetic service I have ever recieved.", "Such a horrible app. I have purchased swiggy one membership. Few days back I've placed an order and the restaurant has cancelled my order with reason that item is out of stock then again after some time I've placed another order from the same restaurant with a thought that might be this time the item is available but again the same thing happened they cancelled my order again without taking my permission. They were keep calling me I received no. Of calls just for the cancellation.", 'Very shameless app you can see on paly store Worst app, there resturent serves ins

In [107]:
# Process data
for date, reviews in daily_data.items():
    agent.process_daily_batch(pd.to_datetime(date).date(), reviews)

# Get trend report on a specific date (rolling window T-30 to T)
trend_report_df = agent.get_trend_report('2022-08-31')

In [108]:
pd.set_option("display.max_rows",None)

In [109]:
trend_report_df

,2022-08-01,2022-08-02,2022-08-03,2022-08-04,2022-08-05,2022-08-06,2022-08-07,2022-08-08,2022-08-09,2022-08-10,2022-08-11,2022-08-12,2022-08-13,2022-08-14,2022-08-15,2022-08-16,2022-08-17,2022-08-18,2022-08-19,2022-08-20,2022-08-21,2022-08-22,2022-08-23,2022-08-24,2022-08-25,2022-08-26,2022-08-27,2022-08-28,2022-08-29,2022-08-30,2022-08-31
Topic,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Delivery issue,0,0,0,0,0,0,2,0,1,0,0,1,0,0,0,0,0,0,2,3,2,0,1,0,2,0,1,1,1,1,2
Food stale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
Delivery partner rude,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Maps not working properly,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
Instamart should be open all night,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Bring back 10 minute bolt delivery,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Payment issue,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Customer support unresponsive,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
new_topic 1: Nice app 👌 but it delivers,9,10,9,10,10,10,8,10,9,10,10,9,10,10,9,10,10,9,6,7,7,10,8,9,8,9,8,7,9,8,5


In [98]:
trend_report_df.to_csv("Output.csv")

In [114]:
# Re-initialize agent for a smaller window matching your data:
agent = TopicLimiterReviewTrendAgent(
    seed_topics=seed_topics,
    similarity_threshold=0.6,
    window_days=31  # match your data's time span
)

# Re-process data
for date, reviews in daily_data.items():
    agent.process_daily_batch(pd.to_datetime(date).date(), reviews)

# Find correct last date and generate report
# last_date = max(pd.to_datetime(d).date() for d in daily_data)
trend_report_df = agent.get_trend_report('2022-08-31')

# Filter and show only nonzero topics
trend_report_df = trend_report_df[trend_report_df.sum(axis=1) > 0]
# print(trend_report_df)


In [115]:
trend_report_df

,2022-08-01,2022-08-02,2022-08-03,2022-08-04,2022-08-05,2022-08-06,2022-08-07,2022-08-08,2022-08-09,2022-08-10,2022-08-11,2022-08-12,2022-08-13,2022-08-14,2022-08-15,2022-08-16,2022-08-17,2022-08-18,2022-08-19,2022-08-20,2022-08-21,2022-08-22,2022-08-23,2022-08-24,2022-08-25,2022-08-26,2022-08-27,2022-08-28,2022-08-29,2022-08-30,2022-08-31
Topic,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Delivery issue,0,0,0,0,0,0,2,0,1,0,0,1,0,0,0,0,0,0,2,3,2,0,1,0,2,0,1,1,1,1,2
Food stale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0
Delivery partner rude,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
Maps not working properly,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
new_topic 1: Nice app 👌 but it delivers,9,10,9,10,10,10,8,10,9,10,10,9,10,10,9,10,10,9,6,7,7,10,8,9,8,9,8,7,9,8,5
new_topic 20: This update is bakwas Colour less,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
new_topic 81: Tremendous efforts! More items can be,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
"new_topic 109: made college life real easy, i",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0
"new_topic 111: Not that good, resolution is slow",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
